In [1]:
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import pandas as pd
from plotnine import *
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import sys
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

In [2]:
#read the data
data = pd.read_csv("f7664209-d97b-4f9b-a7bd-e1fef352177c_Data.csv")

print(data.head())

   Time Time Code    Country Name Country Code  \
0  1992    YR1992     Afghanistan          AFG   
1  1992    YR1992         Albania          ALB   
2  1992    YR1992         Algeria          DZA   
3  1992    YR1992  American Samoa          ASM   
4  1992    YR1992         Andorra          AND   

  GDP per capita, PPP (constant 2017 international $) [NY.GDP.PCAP.PP.KD]  \
0                                                 ..                        
1                                   3275.44433583801                        
2                                   8383.77024779121                        
3                                                 ..                        
4                                                 ..                        

  CO2 emissions (metric tons per capita) [EN.ATM.CO2E.PC]  
0                                 0.0961965810608727       
1                                  0.774724910911141       
2                                   2.96498636282543    

In [3]:
data.columns

Index(['Time', 'Time Code', 'Country Name', 'Country Code',
       'GDP per capita, PPP (constant 2017 international $) [NY.GDP.PCAP.PP.KD]',
       'CO2 emissions (metric tons per capita) [EN.ATM.CO2E.PC]'],
      dtype='object')

In [51]:
len(data)

4516

In [4]:
data.drop(['Time Code','Country Code'],axis=1,inplace=True)

In [5]:
cols = {'GDP per capita, PPP (constant 2017 international $) [NY.GDP.PCAP.PP.KD]':'gdppc',
          'CO2 emissions (metric tons per capita) [EN.ATM.CO2E.PC]':'co2pc',
        'Country Name':'country',
       'Time':'year'}
data.rename(columns=cols, inplace=True)

In [6]:
data.columns

Index(['year', 'country', 'gdppc', 'co2pc'], dtype='object')

In [7]:
data.country=data.country.astype("category")
data.year=data.year.astype("category")

In [8]:
data.head()

,year,country,gdppc,co2pc
0,1992,Afghanistan,..,0.0961965810608727
1,1992,Albania,3275.44433583801,0.774724910911141
2,1992,Algeria,8383.77024779121,2.96498636282543
3,1992,American Samoa,..,..
4,1992,Andorra,..,6.91205338948512


In [9]:
data.replace('..',np.NaN,inplace=True)

In [10]:
data.isnull().sum()

year         3
country      5
gdppc      848
co2pc      848
dtype: int64

In [11]:
data.groupby('country').agg(lambda x: x.isnull().sum()).sort_values(['gdppc']+['co2pc'],ascending=False).head(60)

,year,gdppc,co2pc
country,,,
American Samoa,0,27,27
Channel Islands,0,27,27
Guam,0,27,27
Isle of Man,0,27,27
Monaco,0,27,27
Northern Mariana Islands,0,27,27
St. Martin (French part),0,27,27
Virgin Islands (U.S.),0,27,27
South Sudan,0,27,22


In [12]:
temp=data.dropna(axis=0,thresh=4)

In [13]:
temp.isnull().sum()

year       0
country    0
gdppc      0
co2pc      0
dtype: int64

In [14]:
temp

,year,country,gdppc,co2pc
1,1992,Albania,3275.44433583801,0.774724910911141
2,1992,Algeria,8383.77024779121,2.96498636282543
5,1992,Angola,5148.25282361678,0.410522931864339
6,1992,Antigua and Barbuda,15497.4647441974,4.0952366180143
7,1992,Argentina,16209.3230774449,3.61928035773818
...,...,...,...,...
5417,2016,Vanuatu,3061.78723983667,0.527000323357166
5419,2016,Vietnam,6767.90249535442,2.05756591685295
5421,2016,West Bank and Gaza,6438.93364028868,0.740606555214825
5423,2016,Zambia,3467.87515599636,0.314182894901441


In [15]:
temp.groupby('country').agg(lambda x: x.isnull().sum()).sort_values(['gdppc']+['co2pc'],ascending=False).head(60)

,year,gdppc,co2pc
country,,,
Afghanistan,0,0.0,0.0
Albania,0,0.0,0.0
Algeria,0,0.0,0.0
Angola,0,0.0,0.0
Antigua and Barbuda,0,0.0,0.0
Argentina,0,0.0,0.0
Armenia,0,0.0,0.0
Aruba,0,0.0,0.0
Australia,0,0.0,0.0


In [16]:
temp.head()

,year,country,gdppc,co2pc
1,1992,Albania,3275.44433583801,0.774724910911141
2,1992,Algeria,8383.77024779121,2.96498636282543
5,1992,Angola,5148.25282361678,0.410522931864339
6,1992,Antigua and Barbuda,15497.4647441974,4.0952366180143
7,1992,Argentina,16209.3230774449,3.61928035773818


In [17]:
temp.columns

Index(['year', 'country', 'gdppc', 'co2pc'], dtype='object')

In [18]:
print(type(temp.gdppc[1]))
print(type(temp.co2pc[1]))

<class 'str'>
<class 'str'>


In [19]:
temp.gdppc=pd.to_numeric(temp.gdppc)
temp.co2pc=pd.to_numeric(temp.co2pc)
temp.year=pd.to_numeric(temp.year)

In [20]:
countries_grouped = temp.groupby("country")

In [21]:
temp["lngdppc"] = countries_grouped["gdppc"].transform(np.log)
temp["lnco2pc"] = countries_grouped["co2pc"].transform(np.log)
temp["d_lngdppc"] = countries_grouped["lngdppc"].transform("diff")
temp["d_lnco2pc"] = countries_grouped["lnco2pc"].transform("diff")

In [22]:
data = temp

In [23]:
temp=temp.dropna(axis=0,thresh=7)

In [24]:
temp.isnull().sum()

year         0
country      0
gdppc        0
co2pc        0
lngdppc      0
lnco2pc      0
d_lngdppc    0
d_lnco2pc    0
dtype: int64

In [25]:
temp.groupby('country').size().sort_values().head(20)

country
Eritrea                      0
Somalia                      0
San Marino                   0
Faroe Islands                0
French Polynesia             0
Puerto Rico                  0
Gibraltar                    0
Syrian Arab Republic         0
Greenland                    0
Guam                         0
Northern Mariana Islands     0
Cuba                         0
Isle of Man                  0
South Sudan                  0
Korea, Dem. People’s Rep.    0
Channel Islands              0
St. Martin (French part)     0
Venezuela, RB                0
Monaco                       0
Virgin Islands (U.S.)        0
dtype: int64

In [26]:
###Dropping contries wiht poor coverage (<16 observations)
countries_to_drop=['Sao Tome and Principe','Sudan','Timor-Leste','Djibouti','Curacao','Cayman Islands',
                   'Sint Maarten (Dutch part)','Kosovo','Montenegro','Nauru','Serbia',
                   'Afghanistan','Turks and Caicos Islands',]
temp=temp.set_index('country').drop(index=countries_to_drop,axis=0)

In [52]:
len(countries_to_drop)

13

In [55]:
temp.groupby('country').size().sort_values().tail(20)

country
Eswatini                24
Ethiopia                24
Fiji                    24
Finland                 24
Gabon                   24
Gambia, The             24
Georgia                 24
Ecuador                 24
Germany                 24
Greece                  24
Grenada                 24
Guatemala               24
Guinea                  24
Guinea-Bissau           24
Guyana                  24
Haiti                   24
Honduras                24
Hong Kong SAR, China    24
Ghana                   24
Zimbabwe                24
dtype: int64

In [28]:
temp.reset_index(inplace=True)


In [29]:
temp.query("year == 2000").head(20)

,country,year,gdppc,co2pc,lngdppc,lnco2pc,d_lngdppc,d_lnco2pc
1194,Albania,2000,5911.956097,0.978175,8.684732,-0.022067,0.073565,0.018584
1195,Algeria,2000,8710.455991,2.830380,9.072279,1.040411,0.023712,-0.060106
1196,Angola,2000,4727.967467,0.581961,8.461251,-0.541351,-0.002683,0.008418
1197,Antigua and Barbuda,2000,18311.013715,4.534545,9.815258,1.511725,0.042374,0.025673
1198,Argentina,2000,18625.288101,3.854992,9.832276,1.349369,-0.018927,-0.044544
1199,Armenia,2000,4048.257817,1.128918,8.306042,0.121259,0.063635,0.145755
1200,Aruba,2000,41022.321554,26.194875,10.621872,3.265564,0.052854,1.056720
1201,Australia,2000,38462.015363,17.200610,10.557426,2.844945,0.026655,0.000600
1202,Austria,2000,46551.459787,7.771971,10.748314,2.050524,0.030795,0.001608
1203,Azerbaijan,2000,4063.471641,3.666271,8.309793,1.299175,0.097045,0.023858


In [43]:
formula="d_lnco2pc ~ d_lngdppc"

###OLS regression 2000
ols_cross_section2000 = smf.ols("d_lnco2pc ~ d_lngdppc + country", temp.query("year == 2000")).fit(cov_type="HC0")

In [44]:
models = [ols_cross_section2000]
names = ["Cross-section regression 2000"]
stargazer = Stargazer(models)
stargazer.rename_covariates(
    {
        "Intercept": "Constant",
        "d_lngdppc": "GDP per capita log change, cumulative coeff.",
        "d_lnco2pc": "CO2 emissions per capita log change, cumulative coeff."
    }
)

stargazer.covariate_order(
    [
        "d_lngdppc",
        "Intercept"
    ]
)

stargazer.custom_columns(names, [1])
HTML(stargazer.render_html())

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 217, but rank is 169


The per capita CO2 emissions were 0.132 p.p. higher, on average, in 2000 for every 1 p.p. increase in GDP per capita, controlling for a country.

In [32]:
temp.query("year == 2015")

,country,year,gdppc,co2pc,lngdppc,lnco2pc,d_lngdppc,d_lnco2pc
3861,Albania,2015,11916.422315,1.602648,9.385673,0.471657,0.024857,-0.170233
3862,Algeria,2015,11696.963757,3.854557,9.367085,1.349256,0.015878,0.031293
3863,Angola,2015,8036.410610,1.240245,8.991738,0.215309,-0.024997,-0.294360
3864,Antigua and Barbuda,2015,18595.084904,5.839546,9.830653,1.764653,0.026751,0.023345
3865,Argentina,2015,23933.886613,4.664011,10.083051,1.539876,0.016165,0.016382
...,...,...,...,...,...,...,...,...
4032,Vanuatu,2015,3037.784742,0.486896,8.018884,-0.719706,-0.025446,-0.181224
4033,Vietnam,2015,6438.260271,2.032108,8.770014,0.709073,0.054217,0.107750
4034,West Bank and Gaza,2015,6048.976597,0.704186,8.707644,-0.350712,0.013633,0.034828
4035,Zambia,2015,3443.555206,0.285428,8.144260,-1.253766,-0.001883,-0.024175


In [33]:
formula="d_lnco2pc ~ d_lngdppc"

###Pooled regression 2015
ols_pooled2015 = smf.ols("d_lnco2pc ~ d_lngdppc + country", temp.query("year == 2015")).fit(cov_type="HC0")

In [34]:
models = [ols_pooled2015]
names = ["Pooled regression 2015"]
stargazer = Stargazer(models)
stargazer.rename_covariates(
    {
        "Intercept": "Constant",
        "d_lngdppc": "GDP per capita log change, cumulative coeff.",
        "d_lnco2pc": "CO2 emissions per capita log change, cumulative coeff."
    }
)

stargazer.covariate_order(
    [
        "d_lngdppc",
        "Intercept"
    ]
)

stargazer.custom_columns(names, [1])
HTML(stargazer.render_html())

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 217, but rank is 171


The per capita CO2 emissions were 0.149 p.p. higher, on average, in 2015 for every 1 p.p. increase in GDP per capita, controlling for a country.

In [35]:
# Fd, time trend, no lags

model = smf.wls("d_lnco2pc ~ d_lngdppc + year", temp)
fd_lm = model.fit(
    cov_type="cluster",
    cov_kwds={"groups": temp.loc[model.data.row_labels, "country"]},
)
fd_lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:              d_lnco2pc   R-squared:                       0.075
Model:                            WLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     18.55
Date:                Wed, 31 Mar 2021   Prob (F-statistic):           4.87e-08
Time:                        14:43:35   Log-Likelihood:                 2858.4
No. Observations:                4213   AIC:                            -5711.
Df Residuals:                    4210   BIC:                            -5692.
Df Model:                           2                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1407      0.489      0.288      0.773      -0.817       1.098
d_lngdppc      0.6142      0.102      6.046      0.000       0.415       0.813
year       -7.012e-05      0.000     -0.288      0.774      -0.001       0.000
==============================================================================
Omnibus:                     1844.627   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           166438.455
Skew:                           1.162   Prob(JB):                         0.00
Kurtosis:                      33.704   Cond. No.                     5.84e+05
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The condition number is large, 5.84e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

A 1 p.p. increase in GDP per capita tend to be followed by 0.61 p.p. increase in CO2 emissions per capita, on average, in the data, relative to the country trend.

In [36]:
# FD, time trend, 2 year lags

fd_lm_5_cumul_trend_formula = "d_lnco2pc ~ d_lngdppc.shift(0) + d_lngdppc.shift(1) + \
d_lngdppc.shift(2)+ year"

model = smf.wls(
    fd_lm_5_cumul_trend_formula,
    temp
)

fd_lm_5_cumul_trend = model.fit(
    cov_type="cluster",
    cov_kwds={"groups":  temp.loc[model.data.row_labels, "country"]},
)

fd_lm_5_cumul_trend.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:              d_lnco2pc   R-squared:                       0.075
Model:                            WLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     9.375
Date:                Wed, 31 Mar 2021   Prob (F-statistic):           6.77e-07
Time:                        14:43:35   Log-Likelihood:                 2857.1
No. Observations:                4211   AIC:                            -5704.
Df Residuals:                    4206   BIC:                            -5672.
Df Model:                           4                                         
Covariance Type:              cluster                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.1525      0.490      0.312      0.755      -0.807       1.112
d_lngdppc.shift(0)     0.6156      0.102      6.046      0.000       0.416       0.815
d_lngdppc.shift(1)    -0.0222      0.023     -0.966      0.334      -0.067       0.023
d_lngdppc.shift(2)     0.0238      0.027      0.879      0.380      -0.029       0.077
year               -7.603e-05      0.000     -0.311      0.756      -0.001       0.000
==============================================================================
Omnibus:                     1842.843   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           166181.820
Skew:                           1.161   Prob(JB):                         0.00
Kurtosis:                      33.688   Cond. No.                     5.85e+05
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The condition number is large, 5.85e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

A 1 p.p. increase in GDP per capita tends to be followed by 0.62 p.p. cumulative increase in CO2 emissions per capita, on average, within two years in the data, relative to the country trend.

In [37]:
# FD, time trend, 6 year lags

fd_lm_5_cumul_trend_formula = "d_lnco2pc ~ d_lngdppc.shift(0) + d_lngdppc.shift(1) + \
d_lngdppc.shift(2)+d_lngdppc.shift(3)+d_lngdppc.shift(4)+d_lngdppc.shift(5)+d_lngdppc.shift(6)+year"

model = smf.wls(
    fd_lm_5_cumul_trend_formula,
    temp
)

fd_lm_5_cumul_trend = model.fit(
    cov_type="cluster",
    cov_kwds={"groups":  temp.loc[model.data.row_labels, "country"]},
)

fd_lm_5_cumul_trend.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:              d_lnco2pc   R-squared:                       0.076
Model:                            WLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     4.797
Date:                Wed, 31 Mar 2021   Prob (F-statistic):           2.35e-05
Time:                        14:43:35   Log-Likelihood:                 2875.0
No. Observations:                4207   AIC:                            -5732.
Df Residuals:                    4198   BIC:                            -5675.
Df Model:                           8                                         
Covariance Type:              cluster                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.2176      0.479      0.454      0.650      -0.722       1.157
d_lngdppc.shift(0)     0.6194      0.103      5.991      0.000       0.417       0.822
d_lngdppc.shift(1)    -0.0162      0.023     -0.692      0.489      -0.062       0.030
d_lngdppc.shift(2)     0.0237      0.026      0.910      0.363      -0.027       0.075
d_lngdppc.shift(3)    -0.0340      0.030     -1.119      0.263      -0.094       0.026
d_lngdppc.shift(4)    -0.0269      0.025     -1.080      0.280      -0.076       0.022
d_lngdppc.shift(5)    -0.0080      0.031     -0.261      0.794      -0.068       0.052
d_lngdppc.shift(6)    -0.0116      0.028     -0.416      0.678      -0.066       0.043
year                  -0.0001      0.000     -0.450      0.653      -0.001       0.000
==============================================================================
Omnibus:                     1892.331   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           168685.249
Skew:                           1.224   Prob(JB):                         0.00
Kurtosis:                      33.924   Cond. No.                     5.85e+05
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The condition number is large, 5.85e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

A 1 p.p. increase in GDP per capita tends to be followed by 0.55 p.p. increase in CO2 emissions per capita, on average, within six years in the data, relative to the country trend.

In [38]:
###Panel regression
from linearmodels import PanelOLS

In [39]:
data = data.set_index(['country', 'year'])

In [40]:
data.tail()

,,gdppc,co2pc,lngdppc,lnco2pc,d_lngdppc,d_lnco2pc
country,year,,,,,,
Vanuatu,2016,3061.787240,0.527000,8.026754,-0.640554,0.007870,0.079151
Vietnam,2016,6767.902495,2.057566,8.819946,0.721524,0.049933,0.012450
West Bank and Gaza,2016,6438.933640,0.740607,8.770118,-0.300286,0.062474,0.050427
Zambia,2016,3467.875156,0.314183,8.151297,-1.157780,0.007038,0.095986
Zimbabwe,2016,2934.733650,0.782777,7.984372,-0.244908,-0.007967,-0.130197


In [41]:
fe = PanelOLS.from_formula(
    "lnco2pc ~ lngdppc + TimeEffects + EntityEffects",
    data
).fit(cov_type="clustered", cluster_entity=True)

In [42]:
fe.summary

Dep. Variable:,lnco2pc,R-squared:,0.2562
Estimator:,PanelOLS,R-squared (Between):,-12.225
No. Observations:,4516,R-squared (Within):,0.3598
Date:,"Wed, Mar 31 2021",R-squared (Overall):,-12.315
Time:,14:43:36,Log-likelihood,466.52
Cov. Estimator:,Clustered,,
,,F-statistic:,1480.9
Entities:,217,P-value,0.0000
Avg Obs:,20.811,Distribution:,"F(1,4300)"
Min Obs:,0.0000,,
Max Obs:,25.000,F-statistic (robust):,22.726


Per capita CO2 emissions tend to be 0.75 p.p. higher, on average, for a 1 p.p. increase in per capita GDP within the data, compared to mean per capita CO2 emissions for a country and within a given year.

The long difference regression

In [49]:
temp.query("year == 1993 | year == 2016")

,country,year,gdppc,co2pc,lngdppc,lnco2pc,d_lngdppc,d_lnco2pc
0,Albania,1993,3610.520633,0.723790,8.191607,-0.323254,0.097398,-0.068006
1,Algeria,1993,8027.376688,2.974812,8.990613,1.090181,-0.043440,0.003309
2,Angola,1993,3788.508107,0.441721,8.239728,-0.817077,-0.306685,0.073247
3,Antigua and Barbuda,1993,15991.764201,4.125424,9.679829,1.417169,0.031397,0.007344
4,Argentina,1993,17312.030499,3.471494,9.759157,1.244585,0.065815,-0.041690
...,...,...,...,...,...,...,...,...
4208,Vanuatu,2016,3061.787240,0.527000,8.026754,-0.640554,0.007870,0.079151
4209,Vietnam,2016,6767.902495,2.057566,8.819946,0.721524,0.049933,0.012450
4210,West Bank and Gaza,2016,6438.933640,0.740607,8.770118,-0.300286,0.062474,0.050427
4211,Zambia,2016,3467.875156,0.314183,8.151297,-1.157780,0.007038,0.095986


In [50]:
# Long difference

model = smf.wls("d_lnco2pc ~ d_lngdppc", temp.query("year == 1993 | year == 2016"))
longd_lm = model.fit(
    cov_type="cluster",
    cov_kwds={"groups": temp.loc[model.data.row_labels, "country"]},
)
longd_lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:              d_lnco2pc   R-squared:                       0.115
Model:                            WLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     19.93
Date:                Wed, 31 Mar 2021   Prob (F-statistic):           1.43e-05
Time:                        14:48:07   Log-Likelihood:                 283.89
No. Observations:                 340   AIC:                            -563.8
Df Residuals:                     338   BIC:                            -556.1
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0051      0.006      0.845      0.398      -0.007       0.017
d_lngdppc      0.6812      0.153      4.464      0.000       0.382       0.980
==============================================================================
Omnibus:                      104.034   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2539.289
Skew:                          -0.636   Prob(JB):                         0.00
Kurtosis:                      16.328   Cond. No.                         18.0
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

A 1 p.p. increase in GDP per capita tends to be followed by 0.68 p.p. increase in CO2 emissions per capita, on average, between 1993 and 2016 in the data.